# Final Data Analysis

## Imports and Setup

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

In [2]:
df_relevant = pd.read_excel('results/final/final_results.xlsx', sheet_name='relevant_papers')
df_all_papers = pd.read_csv('results/ICSE_all_papers.csv')

print("Relevant Shape:", df_relevant.shape)
print("All Papers Shape:", df_all_papers.shape)

Relevant Shape: (177, 27)
All Papers Shape: (692, 11)


### Creating Combined and Non-Relevant Dataframes

In [3]:
# Relevant Paper List
relevant_papers = df_relevant['title'].tolist()

# Remove relevant papers from all papers to create final non-relevant set
df_non_relevant = df_all_papers[~df_all_papers['title'].isin(relevant_papers)].copy()

# Make sure all non-relevant papers are marked as such
df_non_relevant['relevant'] = False

# Re-order columns to match relevant dataframe
common_columns = ['reviewer', 'relevant', 'year', 'title', 'authors', 'url', 'abstract', 'artifact_available', 'artifact_reusable', 'artifact_functional', 'ai']
df_non_relevant = df_non_relevant[common_columns]

# Add extra columns: 'task', 'non_llm_approaches', 'models_open_closed', 'num_models', 'model_families', 'model_scale', 'model_size_free_text', 'model_sizes_reported', 'model_config', 'dataset_type', 'programming_language', 'cost', 'cost_free_text', 'artefact_manual', 'contamination', 'contamination_free_text'
extra_columns = ['task', 'non_llm_approaches', 'models_open_closed', 'num_models', 'model_families', 'model_scale', 'model_size_free_text', 'model_sizes_reported', 'model_config', 'dataset_type', 'programming_language', 'cost', 'cost_free_text', 'artefact_manual', 'contamination', 'contamination_free_text']
for col in extra_columns:
    df_non_relevant[col] = None

# Combine relevant and non-relevant dataframes
df_combined = pd.concat([df_relevant, df_non_relevant], ignore_index=True)

In [4]:
print("Combined Shape:", df_combined.shape)
print("Num Unique Papers:", df_combined['title'].nunique())
print("Num Relevant Papers in Combined DF:", df_combined[df_combined['relevant'] == True].shape[0])
print("Num Unique Papers in Relevant DF:", df_relevant['title'].nunique()) # Should match number above

Combined Shape: (692, 27)
Num Unique Papers: 692
Num Relevant Papers in Combined DF: 177
Num Unique Papers in Relevant DF: 177


# Analysis

We now have access to four dataframes for analysis:

- `df_combined`: contains all papers and final columns from our spreadsheet (non-relevant papers just have None values in the fields we completed for the relevant papers)
- `df_relevant`: contains all relevant papers as rows and the final columns we intend to use for analysis
- `df_non_relevant`: contains all non-relevant papers. Our finals columns are present but all filled with None values as we didn't perform data extraction for these papers.

## Initial Analysis

### Number of Papers at Each Stage

In [5]:
# Total numbers
print("Total Papers in Combined DF:", df_combined.shape[0])

print("\nTotal Papers from AI Keywords:")
print(df_combined['ai'].value_counts())

print("\nTotal Relevant Papers:")
print(df_combined['relevant'].value_counts())

Total Papers in Combined DF: 692

Total Papers from AI Keywords:
ai
False    388
True     304
Name: count, dtype: int64

Total Relevant Papers:
relevant
False    515
True     177
Name: count, dtype: int64


In [6]:
# Per Year
print("Papers Per Year:")
print(df_combined.groupby('year')['title'].nunique())

print("\nAI Papers Per Year:")
print(df_combined[df_combined['ai'] == True].groupby('year')['title'].nunique())

print("\nRelevant Papers Per Year:")
print(df_combined[df_combined['relevant'] == True].groupby('year')['title'].nunique())

Papers Per Year:
year
2023    210
2024    236
2025    246
Name: title, dtype: int64

AI Papers Per Year:
year
2023     59
2024     99
2025    146
Name: title, dtype: int64

Relevant Papers Per Year:
year
2023    32
2024    55
2025    90
Name: title, dtype: int64


### Numbers of Relevant Papers (Papers with LLM-based Empirical Studies)

In [7]:
print("Overall Relevant vs Non-Relevant Counts:")
print(df_combined["relevant"].value_counts())

print("\n\nRelevant Papers by Year:")
print(df_relevant["year"].value_counts())

Overall Relevant vs Non-Relevant Counts:
relevant
False    515
True     177
Name: count, dtype: int64


Relevant Papers by Year:
year
2025    90
2024    55
2023    32
Name: count, dtype: int64


### Geo-location of SE Research Institutions

# RQ1 - What Tasks are being tackled in LLM SE studies, and are they fairly evaluated against existing non-LLM techniques?

DF Columns to use:
- 'task' (short-text)
- 'non_llm_approaches' (bool)
- 'dataset_type' (short-text)
- programming_language (short-text)

In [8]:
df_relevant['task_list'] = df_relevant['task'].apply(
    lambda x: [task.strip() for task in str(x).split(';')] if pd.notna(x) else []
)

print(df_relevant['task_list'].explode().value_counts())

task_list
code generation                                   26
program repair                                    24
test generation                                   23
vulnerability detection                           16
bug detection                                     12
code translation                                   8
clone detection                                    6
code completion                                    6
code summarisation                                 6
code comprehension                                 5
type detection                                     5
log parsing                                        5
code search                                        5
fuzzing                                            4
bug reproduction                                   3
commit message generation                          3
fault localisation                                 3
code memorisation detection                        3
test repair                         

In [9]:
# Looking at how many papers cover multiple tasks
df_relevant['num_tasks'] = df_relevant['task_list'].apply(len)

print("Distribution of number of tasks per paper:")
print(df_relevant['num_tasks'].value_counts().sort_index())
print(f"\nPapers covering multiple tasks: {(df_relevant['num_tasks'] > 1).sum()}")
print(f"Percentage covering multiple tasks: {(df_relevant['num_tasks'] > 1).mean()*100:.1f}%")

Distribution of number of tasks per paper:
num_tasks
1     157
2      11
3       5
4       3
13      1
Name: count, dtype: int64

Papers covering multiple tasks: 20
Percentage covering multiple tasks: 11.3%


In [10]:
print(df_combined['non_llm_approaches'].value_counts())

non_llm_approaches
True                                                                                                                                                                                                              92
False                                                                                                                                                                                                             59
FALSE (says TransCoder is not llm but a language modeling appraoch using transformers and seq2seq)                                                                                                                 1
TRUE (FTLR, COMET, VSM, LSI,ArDoCode)                                                                                                                                                                              1
TRUE (AEL, Drain)                                                                                                                

# RQ2 - What models are being used?

DF Columns to use:
- 'models_open_closed' (open/closed/both)
- 'num_models' (int)
- 'model_families' (list of short text)
- 'model_sizes_reported' (NA/none/some/full - currently unfinished)
- 'model_scale' (currently unfinished)

### Open vs. Closed

In [11]:
df_relevant['models_open_closed'].value_counts()

models_open_closed
open      71
both      65
closed    41
Name: count, dtype: int64

In [12]:
total = df_relevant.shape[0]
number_open = df_relevant[~(df_relevant['models_open_closed'] == 'closed')].shape[0]
number_closed = df_relevant[~(df_relevant['models_open_closed'] == 'open')].shape[0]

print(f"Open models in {number_open} out of {total} papers ({(number_open/total)*100:.1f}%)")
print(f"Closed models in {number_closed} out of {total} papers ({(number_closed/total)*100:.1f}%)")

Open models in 136 out of 177 papers (76.8%)
Closed models in 106 out of 177 papers (59.9%)


In [13]:
total_2023 = df_relevant[df_relevant["year"] == 2023].shape[0]
number_open_2023 = df_relevant[(df_relevant["year"] == 2023) & ~(df_relevant['models_open_closed'] == 'closed')].shape[0]
number_open_only_2023 = df_relevant[(df_relevant["year"] == 2023) & (df_relevant['models_open_closed'] == 'open')].shape[0]
number_closed_2023 = df_relevant[(df_relevant["year"] == 2023) & ~(df_relevant['models_open_closed'] == 'open')].shape[0]
number_closed_only_2023 = df_relevant[(df_relevant["year"] == 2023) & (df_relevant['models_open_closed'] == 'closed')].shape[0]
number_both_2023 = df_relevant[(df_relevant["year"] == 2023) & (df_relevant['models_open_closed'] == 'both')].shape[0]

print(f"2023 Papers:")
print(f"Only open models in {number_open_only_2023} out of {total_2023} papers ({(number_open_only_2023/total_2023)*100:.1f}%)")
print(f"Only closed models in {number_closed_only_2023} out of {total_2023} papers ({(number_closed_only_2023/total_2023)*100:.1f}%)")
print(f"Open models in {number_open_2023} out of {total_2023} papers ({(number_open_2023/total_2023)*100:.1f}%)")
print(f"Closed models in {number_closed_2023} out of {total_2023} papers ({(number_closed_2023/total_2023)*100:.1f}%)")
print(f"Both model types in {number_both_2023} out of {total_2023} papers ({(number_both_2023/total_2023)*100:.1f}%)")

total_2024 = df_relevant[df_relevant["year"] == 2024].shape[0]
number_open_2024 = df_relevant[(df_relevant["year"] == 2024) & ~(df_relevant['models_open_closed'] == 'closed')].shape[0]
number_open_only_2024 = df_relevant[(df_relevant["year"] == 2024) & (df_relevant['models_open_closed'] == 'open')].shape[0]
number_closed_2024 = df_relevant[(df_relevant["year"] == 2024) & ~(df_relevant['models_open_closed'] == 'open')].shape[0]
number_closed_only_2024 = df_relevant[(df_relevant["year"] == 2024) & (df_relevant['models_open_closed'] == 'closed')].shape[0]
number_both_2024 = df_relevant[(df_relevant["year"] == 2024) & (df_relevant['models_open_closed'] == 'both')].shape[0]

print(f"\n2024 Papers:")
print(f"Only open models in {number_open_only_2024} out of {total_2024} papers ({(number_open_only_2024/total_2024)*100:.1f}%)")
print(f"Only closed models in {number_closed_only_2024} out of {total_2024} papers ({(number_closed_only_2024/total_2024)*100:.1f}%)")
print(f"Open models in {number_open_2024} out of {total_2024} papers ({(number_open_2024/total_2024)*100:.1f}%)")
print(f"Closed models in {number_closed_2024} out of {total_2024} papers ({(number_closed_2024/total_2024)*100:.1f}%)")
print(f"Both model types in {number_both_2024} out of {total_2024} papers ({(number_both_2024/total_2024)*100:.1f}%)")

total_2025 = df_relevant[df_relevant["year"] == 2025].shape[0]
number_open_2025 = df_relevant[(df_relevant["year"] == 2025) & ~(df_relevant['models_open_closed'] == 'closed')].shape[0]
number_open_only_2025 = df_relevant[(df_relevant["year"] == 2025) & (df_relevant['models_open_closed'] == 'open')].shape[0]
number_closed_2025 = df_relevant[(df_relevant["year"] == 2025) & ~(df_relevant['models_open_closed'] == 'open')].shape[0]
number_closed_only_2025 = df_relevant[(df_relevant["year"] == 2025) & (df_relevant['models_open_closed'] == 'closed')].shape[0]
number_both_2025 = df_relevant[(df_relevant["year"] == 2025) & (df_relevant['models_open_closed'] == 'both')].shape[0]

print(f"\n2025 Papers:")
print(f"Only open models in {number_open_only_2025} out of {total_2025} papers ({(number_open_only_2025/total_2025)*100:.1f}%)")
print(f"Only closed models in {number_closed_only_2025} out of {total_2025} papers ({(number_closed_only_2025/total_2025)*100:.1f}%)")
print(f"Open models in {number_open_2025} out of {total_2025} papers ({(number_open_2025/total_2025)*100:.1f}%)")
print(f"Closed models in {number_closed_2025} out of {total_2025} papers ({(number_closed_2025/total_2025)*100:.1f}%)")
print(f"Both model types in {number_both_2025} out of {total_2025} papers ({(number_both_2025/total_2025)*100:.1f}%)")

print("Concerning Trend: Increasing use of closed models over time! Particularly papers that feature only closed/commercial models.")

2023 Papers:
Only open models in 22 out of 32 papers (68.8%)
Only closed models in 4 out of 32 papers (12.5%)
Open models in 28 out of 32 papers (87.5%)
Closed models in 10 out of 32 papers (31.2%)
Both model types in 6 out of 32 papers (18.8%)

2024 Papers:
Only open models in 25 out of 55 papers (45.5%)
Only closed models in 13 out of 55 papers (23.6%)
Open models in 42 out of 55 papers (76.4%)
Closed models in 30 out of 55 papers (54.5%)
Both model types in 17 out of 55 papers (30.9%)

2025 Papers:
Only open models in 24 out of 90 papers (26.7%)
Only closed models in 24 out of 90 papers (26.7%)
Open models in 66 out of 90 papers (73.3%)
Closed models in 66 out of 90 papers (73.3%)
Both model types in 42 out of 90 papers (46.7%)
Concerning Trend: Increasing use of closed models over time! Particularly papers that feature only closed/commercial models.


In [14]:
print("Number of papers with X models:")
df_relevant['num_models'].value_counts()

Number of papers with X models:


num_models
1     48
3     24
2     23
5     17
4     17
6     13
7      9
8      7
9      5
10     4
11     3
16     2
19     2
15     1
12     1
37     1
Name: count, dtype: int64

In [15]:
df_relevant['model_families_list'] = df_relevant['model_families'].apply(
    lambda x: [model_family.strip() for model_family in str(x).split(';')] if pd.notna(x) else []
)

print("Overall Paper counts per model family:")
print(df_relevant['model_families_list'].explode().value_counts())

Overall Paper counts per model family:
model_families_list
GPT-4                           47
GPT-3.5                         44
CodeBERT                        34
CodeLlama                       26
CodeT5                          22
CodeGen                         19
StarCoder                       18
GraphCodeBERT                   18
Llama                           17
RoBERTa                         15
ChatGPT                         14
BERT                            13
DeepSeekCoder                   12
UniXcoder                       11
Codex                           10
InCoder                          9
T5                               8
Claude                           7
Gemini                           7
ChatGLM                          6
DeepSeek                         6
CodeQwen                         6
GPT-3                            6
UnixCoder                        6
PLBART                           6
CodeGPT                          5
CodeParrot                     

In [16]:
df_relevant[df_relevant['year'] == 2023]['model_families_list'].explode().value_counts()

model_families_list
CodeBERT            11
RoBERTa              8
BERT                 7
CodeT5               7
Codex                5
T5                   5
GraphCodeBERT        4
DistilBERT           3
PLBART               2
GPT-J                2
CodeGen              2
BART                 2
InCoder              2
GPT-Neo              2
XLNet                1
MiniLM               1
ELECTRA              1
ALBERT               1
RepresentThemAll     1
seBERT               1
Code-davinci         1
Curie                1
Davinci              1
T5-learning          1
CodeGPT              1
JavaBERT             1
DOBF                 1
CuBERT               1
ProphetNet-Code      1
SPT-Code             1
CoTexT               1
C-BERT               1
GPT-C                1
CugLM                1
TreeBERT             1
GPT-2                1
SynCoBERT            1
DeepDebug            1
UniXcoder            1
GPT-NeoX             1
PLBart               1
GPT-3                1
CodeParrot    

# RQ3 - How well do authors tackle the problem of data leakage/contamination?

DF Columns to use:
- contamination (bool)
- contamination_free_text (free text)

In [17]:
# Overall
total = df_relevant.shape[0]
contamination_reported = df_relevant['contamination'].sum()

print(f"All years: Contamination reported in {contamination_reported} out of {total} papers ({(contamination_reported/total)*100:.1f}%)")

All years: Contamination reported in 58 out of 177 papers (32.8%)


In [18]:
# Per Year
total_2023 = df_relevant[df_relevant['year'] == 2023].shape[0]
contamination_2023 = df_relevant[df_relevant['year'] == 2023]['contamination'].sum()
print(f"2023: Contamination reported in {contamination_2023} out of {total_2023} papers ({(contamination_2023/total_2023)*100:.1f}%)")

total_2024 = df_relevant[df_relevant['year'] == 2024].shape[0]
contamination_2024 = df_relevant[df_relevant['year'] == 2024]['contamination'].sum()
print(f"2024: Contamination reported in {contamination_2024} out of {total_2024} papers ({(contamination_2024/total_2024)*100:.1f}%)")

total_2025 = df_relevant[df_relevant['year'] == 2025].shape[0]
contamination_2025 = df_relevant[df_relevant['year'] == 2025]['contamination'].sum()
print(f"2025: Contamination reported in {contamination_2025} out of {total_2025} papers ({(contamination_2025/total_2025)*100:.1f}%)")

2023: Contamination reported in 6 out of 32 papers (18.8%)
2024: Contamination reported in 14 out of 55 papers (25.5%)
2025: Contamination reported in 38 out of 90 papers (42.2%)


# RQ4 - How replicable are LLM-based studies?

DF Columns to use:
- 'model_config' (short-text list)
- 'artifact_available' (bool)
- 'artifact_reusable' (bool)
- 'artifact_functional' (bool)
- 'artefact_manual' (bool)

## ACM Badge Artifact Availability - Relevant vs. Non-Relevant

In [19]:
# Calculate proportions in the combined dataset
total_relevant = df_relevant.shape[0]
total_non_relevant = df_non_relevant.shape[0]

relevant_with_artifact = df_relevant['artifact_available'].sum()
non_relevant_with_artifact = df_non_relevant['artifact_available'].sum()

prop_relevant = relevant_with_artifact / total_relevant
prop_non_relevant = non_relevant_with_artifact / total_non_relevant

print(f"Proportion of relevant papers with artifact available: {prop_relevant:.2%} ({relevant_with_artifact}/{total_relevant})")
print(f"Proportion of other papers with artifact available: {prop_non_relevant:.2%} ({non_relevant_with_artifact}/{total_non_relevant})")

Proportion of relevant papers with artifact available: 18.64% (33/177)
Proportion of other papers with artifact available: 41.36% (213/515)


In [20]:
# Proportion of relevant papers with artifacts available per year
total_relevant_2023 = df_relevant[df_relevant['year'] == 2023].shape[0]
artifact_avail_2023 = df_relevant[(df_relevant['year'] == 2023) & (df_relevant['artifact_available'])].shape[0]
prop_relevant_2023 = artifact_avail_2023 / total_relevant_2023
print(f"2023 - Proportion of relevant papers with artifact available: {prop_relevant_2023:.2%} ({artifact_avail_2023}/{total_relevant_2023})")

total_relevant_2024 = df_relevant[df_relevant['year'] == 2024].shape[0]
artifact_avail_2024 = df_relevant[(df_relevant['year'] == 2024) & (df_relevant['artifact_available'])].shape[0]
prop_relevant_2024 = artifact_avail_2024 / total_relevant_2024
print(f"2024 - Proportion of relevant papers with artifact available: {prop_relevant_2024:.2%} ({artifact_avail_2024}/{total_relevant_2024})")

total_relevant_2025 = df_relevant[df_relevant['year'] == 2025].shape[0]
artifact_avail_2025 = df_relevant[(df_relevant['year'] == 2025) & (df_relevant['artifact_available'])].shape[0]
prop_relevant_2025 = artifact_avail_2025 / total_relevant_2025
print(f"2025 - Proportion of relevant papers with artifact available: {prop_relevant_2025:.2%} ({artifact_avail_2025}/{total_relevant_2025})")

2023 - Proportion of relevant papers with artifact available: 18.75% (6/32)
2024 - Proportion of relevant papers with artifact available: 16.36% (9/55)
2025 - Proportion of relevant papers with artifact available: 20.00% (18/90)


## Manual Artefact Availability

In [21]:
print(df_relevant["artefact_manual"].value_counts())
print("10 instances of dead links in the 173 relevant papers")

# Per year
print("2023 Artifacts (Manual Checking)")
print(df_relevant[df_relevant['year'] == 2023]['artefact_manual'].value_counts())
print("\n2024 Artifacts (Manual Checking)")
print(df_relevant[df_relevant['year'] == 2024]['artefact_manual'].value_counts())
print("\n2025 Artifacts (Manual Checking)")
print(df_relevant[df_relevant['year'] == 2025]['artefact_manual'].value_counts())

artefact_manual
True     144
False     24
DEAD       9
Name: count, dtype: int64
10 instances of dead links in the 173 relevant papers
2023 Artifacts (Manual Checking)
artefact_manual
True     24
False     5
DEAD      3
Name: count, dtype: int64

2024 Artifacts (Manual Checking)
artefact_manual
True     44
False     8
DEAD      3
Name: count, dtype: int64

2025 Artifacts (Manual Checking)
artefact_manual
True     76
False    11
DEAD      3
Name: count, dtype: int64


# RQ5 - How sustainable is LLM-based SE research?

DF Columns to use:
- 'cost' (short-text list)
- 'cost_free_text' (free text)